# Fluent Python

## Chapter 9 Decorators and Closures

### Decorator란?
- 함수를 인수로 받아 명령을 추가한 뒤 이를 다시 함수의 형태로 반환하는 함수

함수의 내부를 수정하지 않고 기능에 변화를 주고 싶을 때 사용  
일반적으로 함수의 전처리나 후처리에 대한 필요가 있을때 사용  
반복을 줄이고 메소드나 함수의 책임을 확장

간단한 사용법 예시

In [163]:
# decorator
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco 
def target():
    print('running target()')
    
target() # running inner()
print(target)
print()

# target = deco(target) 과 동일
target = deco(target)
target() # running inner()
print(target)

running inner()
<function deco.<locals>.inner at 0x0000026477922680>

running inner()
<function deco.<locals>.inner at 0x0000026476773130>


#### Decorator는 모듈이 load될 때 즉시 시행된다

Decorator를 적용시킨 함수를 활용 하지 않아도, 정의한 것 만으로 실행이 됨.

In [164]:
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

running register(<function f1 at 0x0000026476B356C0>)
running register(<function f2 at 0x00000264779CDE10>)


In [165]:
registry

[<function __main__.f1()>, <function __main__.f2()>]

python3 registration.py의 실행결과

```
running register(<function f1 at 0x000001EC6258ADD0>)
running register(<function f2 at 0x000001EC6258AE60>)
running main()
registry -> [<function f1 at 0x000001EC6258ADD0>, <function f2 at 0x000001EC6258AE60>]
running f1()
running f2()
running f3()
```

In [105]:
import registration

registration.registry

[<function registration.f1()>, <function registration.f2()>]

### 전역 변수와 지역 변수 (Variable Scope Rules)
전역 변수(글로벌 변수)와 지역 변수를 구별 하여 사용해야 함

전역 변수: 함수밖, 전체 범위에서 선언되어 코드 전체에 영향을 주고 받는 변수  
지역 변수: 지역 범위에서만 영향을 주고 받는 변수  


In [166]:
def f1(a):
    print(a)
    print(b)

b = 6
f1(3)

3
6


In [167]:
b = 6 
def f2(a):
    print(a)
    print(b)
    b=9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

f2() 의 body를 컴파일 할 때, b는 이미 지역변수로 설정이 됨.  
고로 ```print(b)``` 에서는 전역 변수 b가 아닌, 지역변수 b를 fetch하려고 하고, 위와같은 error가 남

In [168]:
# 이와 같이 global변수로 설정 해주면 okay
b = 6 
def f3(a):
    global b
    print(a)
    print(b)
    b=9

f3(3)
print(b)

3
6
9


### 스코프(scope)의 개념

**local 스코프**: inner 함수 블록 안에 있는 영역  
**nonlocal 스코프**: outer 함수의 안에 있되, inner 의 밖에 있는 영역  
**global 스코프**: outer 함수 밖의 영역  

```
... (global)
def outer():
    ... (nonlocal)
    def inner():
        ... (local)
```



### Closures
정의: 자신을 둘러싼 스코프(scope)의 상태값을 기억하는 함수

조건
1. 해당 함수는 어떤 함수 내의 중첩된 함수여야 한다.
2. 해당 함수는 자신을 둘러싼(enclose) 함수 내의 상태값을 반드시 참조해야 한다.
3. 해당 함수를 둘러싼 함수는 이 함수를 반환해야 한다.

In [169]:
def make_averager():
    series = [] 
    
    def averager(new_value): # 클로져
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

In [170]:
avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(15))

10.0
10.5
12.0


### The nonlocal Declaration
nonlocal 스코프의 변수를 local에서 사용하고 싶을 때

In [171]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1 # 사실상 count = count + 1 로 count를 assign하고 있음 (local 스코프의 변수로 취급됨)
        total += new_value
        return total / count
    
    return averager

avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(15))


UnboundLocalError: local variable 'count' referenced before assignment

In [172]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total # count, total이 nonlocal 스코프의 변수임을 설정해줌
        count += 1 
        total += new_value
        return total / count
    
    return averager

avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(15))

10.0
10.5
12.0


### Implementing a Simple Decorator (예시)

clock decorator -> 함수의 실행시간과 인수, 결과값을 알려줌


In [174]:
import time
def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        # func 함수의 실행 시간, 어떤 args를 받아서 어떤 결과를 냈는지를 print
        print(f'[execution time: {elapsed:0.8f}s] {name}({arg_str}) -> {result!r}') 
        return result
    return clocked

In [175]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)


In [176]:
print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))


**************************************** Calling snooze(.123)
[execution time: 0.12368530s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[execution time: 0.00000070s] factorial(1) -> 1
[execution time: 0.00001520s] factorial(2) -> 2
[execution time: 0.00002080s] factorial(3) -> 6
[execution time: 0.00002620s] factorial(4) -> 24
[execution time: 0.00003150s] factorial(5) -> 120
[execution time: 0.00003750s] factorial(6) -> 720
6! = 720


In [116]:
# snooze = clock(snooze)를 하고있기에,
print(snooze.__name__)
print(factorial.__name__)

clocked
clocked


In [177]:
import time
import functools
def clock(func):
    @functools.wraps(func) # func로부터 attributes를 copy
    def clocked(*args, **kwargs): 
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = [repr(arg) for arg in args]
        arg_lst.extend(f'{k}={v!r}' for k, v in kwargs.items())
        arg_str = ', '.join(arg_lst)
        print(f'[execution time:{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked


In [178]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    
snooze(.123)
print(snooze.__name__)

[execution time:0.13097180s] snooze(0.123) -> None
snooze


In [179]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

print(fibonacci(6))


[execution time:0.00000050s] fibonacci(0) -> 0
[execution time:0.00000060s] fibonacci(1) -> 1
[execution time:0.00026610s] fibonacci(2) -> 1
[execution time:0.00000030s] fibonacci(1) -> 1
[execution time:0.00000040s] fibonacci(0) -> 0
[execution time:0.00000020s] fibonacci(1) -> 1
[execution time:0.00000820s] fibonacci(2) -> 1
[execution time:0.00001620s] fibonacci(3) -> 2
[execution time:0.00029150s] fibonacci(4) -> 3
[execution time:0.00000020s] fibonacci(1) -> 1
[execution time:0.00000020s] fibonacci(0) -> 0
[execution time:0.00000020s] fibonacci(1) -> 1
[execution time:0.00000700s] fibonacci(2) -> 1
[execution time:0.00001440s] fibonacci(3) -> 2
[execution time:0.00000020s] fibonacci(0) -> 0
[execution time:0.00000020s] fibonacci(1) -> 1
[execution time:0.00000710s] fibonacci(2) -> 1
[execution time:0.00000020s] fibonacci(1) -> 1
[execution time:0.00000030s] fibonacci(0) -> 0
[execution time:0.00000020s] fibonacci(1) -> 1
[execution time:0.00000740s] fibonacci(2) -> 1
[execution ti

In [180]:
@functools.cache # 호출 결과가 이미 캐시되어 있으면 함수를 실행하지 않고 캐시 결과만 반환 (clock에 적용됨)
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

print(fibonacci(6))

[execution time:0.00000100s] fibonacci(0) -> 0
[execution time:0.00000050s] fibonacci(1) -> 1
[execution time:0.00021030s] fibonacci(2) -> 1
[execution time:0.00000080s] fibonacci(3) -> 2
[execution time:0.00022640s] fibonacci(4) -> 3
[execution time:0.00000070s] fibonacci(5) -> 5
[execution time:0.00024220s] fibonacci(6) -> 8
8


### Single Dispatch Generic Functions (예시)

@singledispatch decorator를 이용하여, function을 generic하게 바꿔줌

인수의 type에 따라 다른 함수를 참조하도록 설정 가능

In [181]:
from functools import singledispatch
from collections import abc
import html

# a group of functions to perform the same operation in different ways, \
# depending on the type of the first argument
# 첫번째 인수로 결정되는 entry 함수
@singledispatch 
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

@htmlize.register
def _(text: str) -> str:
    content = html.escape(text).replace('\n', '<br/>\n')
    return f'<p>{content}</p>'

@htmlize.register
def _(seq: abc.Sequence) -> str:
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

@htmlize.register(int)
def _(n) -> str:
    return f'<pre>{n} (0x{n:x})</pre>'


print(htmlize(abs))
print()
print(htmlize('Heimlich & Co.\n - a game'))
print()
print(htmlize([1, 2, 3]))
print()
print(htmlize(42))


<pre>&lt;built-in function abs&gt;</pre>

<p>Heimlich &amp; Co.<br/>
 - a game</p>

<ul>
<li><pre>1 (0x1)</pre></li>
<li><pre>2 (0x2)</pre></li>
<li><pre>3 (0x3)</pre></li>
</ul>

<pre>42 (0x2a)</pre>


### Parameterized Decorators


In [182]:
registry = set()
def register(active=True):
    def decorate(func):
        print('running register'
            f'(active={active})->decorate({func})')
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        
        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')
 
@register()
def f2():
    print('running f2()')
 
def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x0000026477A18B80>)
running register(active=True)->decorate(<function f2 at 0x0000026477922B00>)


In [183]:
print(registry)

{<function f2 at 0x0000026477922B00>}


In [184]:
register()(f3)

running register(active=True)->decorate(<function f3 at 0x0000026477922680>)


<function __main__.f3()>

In [185]:
print(registry)

{<function f2 at 0x0000026477922B00>, <function f3 at 0x0000026477922680>}


In [186]:
register(active=False)(f2)

running register(active=False)->decorate(<function f2 at 0x0000026477922B00>)


<function __main__.f2()>

In [187]:
print(registry)

{<function f3 at 0x0000026477922680>}


### The Parameterized Clock Decorator (예시)


In [188]:
import time
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'
def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals())) # fnt를 사용
            return _result
        return clocked
    return decorate

In [189]:
@clock()
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)
    

[0.12581810s] snooze(0.123) -> None
[0.12287060s] snooze(0.123) -> None
[0.12380960s] snooze(0.123) -> None


In [190]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.130s
snooze(0.123) dt=0.130s
snooze(0.123) dt=0.123s


In [191]:
# class로도 같은 decorator를 반들 수 있음
import time
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'
class clock:
   def __init__(self, fmt=DEFAULT_FMT):
      self.fmt = fmt
      
   def __call__(self, func):
      def clocked(*_args):
         t0 = time.perf_counter()
         _result = func(*_args)
         elapsed = time.perf_counter() - t0
         name = func.__name__
         args = ', '.join(repr(arg) for arg in _args)
         result = repr(_result)
         print(self.fmt.format(**locals()))
         return _result
      return clocked
   
   

In [192]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.124s
snooze(0.123) dt=0.130s
snooze(0.123) dt=0.131s


In [ ]:
# fuctools.partial -> 값을 고정시켜줌 (decorator의 일종)
# 이걸 사용하면 decorator전체 코드를 바꿀 필요가 없음
